# More SQL parsing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mo_sql_parsing as mosp

## SELECT

In [3]:
mosp.parse("SELECT id AS i FROM foo")

{'select': {'value': 'id', 'name': 'i'}, 'from': 'foo'}

In [4]:
mosp.parse("SELECT id AS i, key AS k FROM foo")

{'select': [{'value': 'id', 'name': 'i'}, {'value': 'key', 'name': 'k'}],
 'from': 'foo'}

In [5]:
mosp.parse("SELECT MIN(id) FROM foo")

{'select': {'value': {'min': 'id'}}, 'from': 'foo'}

In [6]:
mosp.parse("SELECT id + (SELECT MIN(id) FROM foo) FROM foo")

{'select': {'value': {'add': ['id',
    {'select': {'value': {'min': 'id'}}, 'from': 'foo'}]}},
 'from': 'foo'}

In [7]:
mosp.parse("SELECT DISTINCT id FROM foo")

{'select_distinct': {'value': 'id'}, 'from': 'foo'}

In [8]:
mosp.parse("SELECT MIN(id) AS m FROM foo")

{'select': {'value': {'min': 'id'}, 'name': 'm'}, 'from': 'foo'}

In [9]:
mosp.parse("SELECT COUNT(DISTINCT id), MIN(id) FROM foo")

{'select': [{'value': {'distinct': True, 'count': 'id'}},
  {'value': {'min': 'id'}}],
 'from': 'foo'}

## Basic predicates

In [10]:
mosp.parse("SELECT * FROM foo, bar WHERE id = fkey")

{'select': '*', 'from': ['foo', 'bar'], 'where': {'eq': ['id', 'fkey']}}

In [11]:
mosp.parse("SELECT * FROM foo, bar WHERE id != fkey")

{'select': '*', 'from': ['foo', 'bar'], 'where': {'neq': ['id', 'fkey']}}

In [12]:
mosp.parse("SELECT * FROM foo WHERE id = 'bar'")

{'select': '*', 'from': 'foo', 'where': {'eq': ['id', {'literal': 'bar'}]}}

In [13]:
mosp.parse("SELECT * FROM foo WHERE id LIKE 'bar'")

{'select': '*', 'from': 'foo', 'where': {'like': ['id', {'literal': 'bar'}]}}

In [14]:
mosp.parse("SELECT * FROM foo WHERE id NOT LIKE 'bar'")

{'select': '*',
 'from': 'foo',
 'where': {'not_like': ['id', {'literal': 'bar'}]}}

In [15]:
mosp.parse("SELECT * FROM foo WHERE id < '1990-01-10'")

{'select': '*',
 'from': 'foo',
 'where': {'lt': ['id', {'literal': '1990-01-10'}]}}

In [16]:
mosp.parse("SELECT * FROM foo WHERE id = 42")

{'select': '*', 'from': 'foo', 'where': {'eq': ['id', 42]}}

## Compound predicates

In [17]:
mosp.parse("SELECT * FROM foo WHERE id = 42 AND key = 'bar'")

{'select': '*',
 'from': 'foo',
 'where': {'and': [{'eq': ['id', 42]}, {'eq': ['key', {'literal': 'bar'}]}]}}

In [18]:
mosp.parse("SELECT * FROM foo WHERE id = 42 OR key = 'bar'")

{'select': '*',
 'from': 'foo',
 'where': {'or': [{'eq': ['id', 42]}, {'eq': ['key', {'literal': 'bar'}]}]}}

In [19]:
mosp.parse("SELECT * FROM foo WHERE id = 42 AND (key = 'bar' OR key = 'baz')")

{'select': '*',
 'from': 'foo',
 'where': {'and': [{'eq': ['id', 42]},
   {'or': [{'eq': ['key', {'literal': 'bar'}]},
     {'eq': ['key', {'literal': 'baz'}]}]}]}}

In [20]:
mosp.parse("SELECT * FROM foo WHERE id = 42 AND NOT (key = 'bar' OR key = 'baz')")

{'select': '*',
 'from': 'foo',
 'where': {'and': [{'eq': ['id', 42]},
   {'not': {'or': [{'eq': ['key', {'literal': 'bar'}]},
      {'eq': ['key', {'literal': 'baz'}]}]}}]}}

## Aliases

In [21]:
mosp.parse("SELECT * FROM foo f, bar b WHERE f.id = b.fkey")

{'select': '*',
 'from': [{'value': 'foo', 'name': 'f'}, {'value': 'bar', 'name': 'b'}],
 'where': {'eq': ['f.id', 'b.fkey']}}

## Subqueries

In [22]:
mosp.parse("SELECT * FROM foo WHERE id IN (SELECT fkey FROM bar)")

{'select': '*',
 'from': 'foo',
 'where': {'in': ['id', {'select': {'value': 'fkey'}, 'from': 'bar'}]}}

## Special predicates

In [23]:
mosp.parse("SELECT * FROM foo, bar WHERE id IN ('bar', 'baz')")

{'select': '*',
 'from': ['foo', 'bar'],
 'where': {'in': ['id', {'literal': ['bar', 'baz']}]}}

In [24]:
mosp.parse("SELECT * FROM foo WHERE id IN ('bar', (SELECT MIN(fkey) FROM bar))")

{'select': '*',
 'from': 'foo',
 'where': {'in': ['id',
   [{'literal': 'bar'},
    {'select': {'value': {'min': 'fkey'}}, 'from': 'bar'}]]}}

In [25]:
mosp.parse("SELECT * FROM foo, bar WHERE id IN ('bar', bar.id, 'biz')")

{'select': '*',
 'from': ['foo', 'bar'],
 'where': {'in': ['id', [{'literal': 'bar'}, 'bar.id', {'literal': 'biz'}]]}}

In [26]:
mosp.parse("SELECT * FROM foo WHERE id IN (SELECT id FROM bar)")

{'select': '*',
 'from': 'foo',
 'where': {'in': ['id', {'select': {'value': 'id'}, 'from': 'bar'}]}}

In [27]:
mosp.parse("SELECT * FROM foo WHERE id BETWEEN 24 AND 42")

{'select': '*', 'from': 'foo', 'where': {'between': ['id', 24, 42]}}

In [28]:
mosp.parse("SELECT * FROM foo WHERE id IS NOT NULL")

{'select': '*', 'from': 'foo', 'where': {'exists': 'id'}}

In [29]:
mosp.parse("SELECT * FROM foo WHERE id IS NULL")

{'select': '*', 'from': 'foo', 'where': {'missing': 'id'}}

## Advanced value manipulations

In [30]:
mosp.parse("SELECT * FROM foo WHERE id = CURRENT_DATE()")

{'select': '*', 'from': 'foo', 'where': {'eq': ['id', {'current_date': {}}]}}

In [31]:
mosp.parse("SELECT * FROM foo WHERE id + '5 days'::interval = 42")

{'select': '*',
 'from': 'foo',
 'where': {'eq': [{'add': ['id',
     {'cast': [{'literal': '5 days'}, {'interval': {}}]}]},
   42]}}

In [32]:
mosp.parse("SELECT id::interval FROM foo")

{'select': {'value': {'cast': ['id', {'interval': {}}]}}, 'from': 'foo'}

## SELECT ... FROM ... JOIN ...

In [33]:
mosp.parse("SELECT * FROM foo JOIN bar")

{'select': '*', 'from': ['foo', {'join': 'bar'}]}

In [34]:
mosp.parse("SELECT * FROM foo JOIN bar ON id = fkey")

{'select': '*', 'from': ['foo', {'join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [35]:
mosp.parse("SELECT * FROM foo f JOIN bar b")

{'select': '*',
 'from': [{'value': 'foo', 'name': 'f'},
  {'join': {'value': 'bar', 'name': 'b'}}]}

In [36]:
mosp.parse("SELECT * FROM foo JOIN bar ON id = fkey AND id = 'baz'")

{'select': '*',
 'from': ['foo',
  {'join': 'bar',
   'on': {'and': [{'eq': ['id', 'fkey']},
     {'eq': ['id', {'literal': 'baz'}]}]}}]}

In [37]:
mosp.parse("SELECT * FROM foo JOIN (SELECT * FROM bar) bar2 ON id = fkey")

{'select': '*',
 'from': ['foo',
  {'join': {'value': {'select': '*', 'from': 'bar'}, 'name': 'bar2'},
   'on': {'eq': ['id', 'fkey']}}]}

In [38]:
mosp.parse("SELECT * FROM foo CROSS JOIN bar ON id = fkey")

{'select': '*',
 'from': ['foo', {'cross join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [39]:
mosp.parse("SELECT * FROM foo FULL JOIN bar ON id = fkey")

{'select': '*',
 'from': ['foo', {'full join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [40]:
mosp.parse("SELECT * FROM foo LEFT JOIN bar ON id = fkey")

{'select': '*',
 'from': ['foo', {'left join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [41]:
mosp.parse("SELECT * FROM foo OUTER JOIN bar ON id = fkey")

{'select': '*',
 'from': ['foo', {'outer join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [42]:
mosp.parse("SELECT * FROM foo RIGHT OUTER JOIN bar ON id = fkey")

{'select': '*',
 'from': ['foo', {'right outer join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [43]:
mosp.parse("SELECT * FROM foo INNER JOIN bar ON id = fkey")

{'select': '*',
 'from': ['foo', {'inner join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [44]:
mosp.parse("SELECT * FROM foo NATURAL JOIN bar")  # THIS IS THE WRONG RESULT!

{'select': '*', 'from': [{'value': 'foo', 'name': 'NATURAL'}, {'join': 'bar'}]}

In [45]:
mosp.parse("SELECT * FROM foo JOIN bar USING id")

{'select': '*', 'from': ['foo', {'join': 'bar', 'using': 'id'}]}

In [46]:
mosp.parse("SELECT * FROM foo JOIN bar USING (id, fkey)")

{'select': '*', 'from': ['foo', {'join': 'bar', 'using': ['id', 'fkey']}]}

## EXPLAIN

In [47]:
mosp.parse("EXPLAIN SELECT * FROM foo, bar WHERE id = fkey")

{'explain': {'select': '*',
  'from': ['foo', 'bar'],
  'where': {'eq': ['id', 'fkey']}}}

In [48]:
mosp.parse("EXPLAIN (ANALYZE, FORMAT JSON) SELECT * FROM foo, bar WHERE id = fkey")

{'analyze': True,
 'format': 'json',
 'explain': {'select': '*',
  'from': ['foo', 'bar'],
  'where': {'eq': ['id', 'fkey']}}}

## Mathematics

In [49]:
mosp.parse("SELECT id + 5 FROM foo")

{'select': {'value': {'add': ['id', 5]}}, 'from': 'foo'}

In [50]:
mosp.parse("SELECT id - 5 FROM foo")

{'select': {'value': {'sub': ['id', 5]}}, 'from': 'foo'}

In [51]:
mosp.parse("SELECT id * 2 FROM foo")

{'select': {'value': {'mul': ['id', 2]}}, 'from': 'foo'}

In [52]:
mosp.parse("SELECT id / 1000 FROM foo")

{'select': {'value': {'div': ['id', 1000]}}, 'from': 'foo'}

In [53]:
mosp.parse("SELECT id % 2 FROM foo")

{'select': {'value': {'mod': ['id', 2]}}, 'from': 'foo'}

In [54]:
mosp.parse("SELECT -id FROM foo")

{'select': {'value': {'neg': 'id'}}, 'from': 'foo'}

## Aggregate Functions

In [55]:
mosp.parse("SELECT COUNT(*) FROM foo")

{'select': {'value': {'count': '*'}}, 'from': 'foo'}

In [56]:
mosp.parse("SELECT COUNT(id) FROM foo")

{'select': {'value': {'count': 'id'}}, 'from': 'foo'}

In [57]:
mosp.parse("SELECT AVG(id) FROM foo")

{'select': {'value': {'avg': 'id'}}, 'from': 'foo'}

In [58]:
mosp.parse("SELECT AVG('bar') FROM foo")

{'select': {'value': {'avg': {'literal': 'bar'}}}, 'from': 'foo'}

In [59]:
mosp.parse("SELECT MIN(id) FROM foo")

{'select': {'value': {'min': 'id'}}, 'from': 'foo'}

In [60]:
mosp.parse("SELECT MAX(id) FROM foo")

{'select': {'value': {'max': 'id'}}, 'from': 'foo'}

In [61]:
mosp.parse("SELECT SUM(id) FROM foo")

{'select': {'value': {'sum': 'id'}}, 'from': 'foo'}

## Function calls

In [62]:
mosp.parse("SELECT CURRENT_DATE() FROM foo WHERE TIMESTAMP() = id")

{'select': {'value': {'current_date': {}}},
 'from': 'foo',
 'where': {'eq': [{'timestamp': {}}, 'id']}}

In [63]:
mosp.parse("SELECT SOUNDEX(UPPER('id'))")

{'select': {'value': {'soundex': {'upper': {'literal': 'id'}}}}}

In [64]:
mosp.parse("SELECT REPEAT('id', 5)")

{'select': {'value': {'repeat': [{'literal': 'id'}, 5]}}}

In [65]:
mosp.parse("SELECT TIME(id) FROM foo")

{'select': {'value': {'time': 'id'}}, 'from': 'foo'}

In [66]:
mosp.parse("SELECT TIME(id OR key OR fkey) FROM foo")

{'select': {'value': {'time': {'or': ['id', 'key', 'fkey']}}}, 'from': 'foo'}

## GROUP BY, ORDER BY, HAVING, LIMIT

In [67]:
mosp.parse("SELECT * FROM foo GROUP BY id")

{'select': '*', 'from': 'foo', 'groupby': {'value': 'id'}}

In [68]:
mosp.parse("SELECT * FROM foo GROUP BY id, key")

{'select': '*', 'from': 'foo', 'groupby': [{'value': 'id'}, {'value': 'key'}]}

In [69]:
mosp.parse("SELECT * FROM foo GROUP BY ALL id")

{'select': '*', 'from': 'foo', 'groupby': {'value': 'ALL', 'name': 'id'}}

In [70]:
mosp.parse("SELECT * FROM foo GROUP BY DISTINCT id")

{'select': '*', 'from': 'foo', 'groupby': {'value': {'distinct': 'id'}}}

In [71]:
mosp.parse("SELECT * FROM foo GROUP BY DISTINCT id, key")

{'select': '*',
 'from': 'foo',
 'groupby': {'value': {'distinct': [{'value': 'id'}, {'value': 'key'}]}}}

In [72]:
mosp.parse("SELECT * FROM foo GROUP BY id HAVING SUM(id) = 42")

{'select': '*',
 'from': 'foo',
 'groupby': {'value': 'id'},
 'having': {'eq': [{'sum': 'id'}, 42]}}

In [73]:
mosp.parse("SELECT * FROM foo GROUP BY id HAVING SUM(id) = 42 AND id = 1")

{'select': '*',
 'from': 'foo',
 'groupby': {'value': 'id'},
 'having': {'and': [{'eq': [{'sum': 'id'}, 42]}, {'eq': ['id', 1]}]}}

In [74]:
mosp.parse("SELECT * FROM foo ORDER BY id")

{'select': '*', 'from': 'foo', 'orderby': {'value': 'id'}}

In [75]:
mosp.parse("SELECT * FROM foo ORDER BY id DESC")

{'select': '*', 'from': 'foo', 'orderby': {'value': 'id', 'sort': 'desc'}}

In [76]:
mosp.parse("SELECT * FROM foo ORDER BY id, key")

{'select': '*', 'from': 'foo', 'orderby': [{'value': 'id'}, {'value': 'key'}]}

In [77]:
mosp.parse("SELECT * FROM foo ORDER BY id DESC, key ASC")

{'select': '*',
 'from': 'foo',
 'orderby': [{'value': 'id', 'sort': 'desc'}, {'value': 'key', 'sort': 'asc'}]}

In [78]:
mosp.parse("SELECT * FROM foo LIMIT 10 OFFSET 20")


{'select': '*', 'from': 'foo', 'limit': 10, 'offset': 20}

## Common Table Expressions


In [79]:
mosp.parse("""
WITH top100_posters AS (
	SELECT owner_user_id, COUNT(*) AS n_answers
	FROM answer, site
	WHERE site.site_name = 'stackoverflow'
		AND answer.site_id = site.site_id
	GROUP BY owner_user_id
	ORDER BY n_answers DESC
	LIMIT 100
)
SELECT account.display_name, top100_posters.n_answers
FROM account, so_user, site, top100_posters
WHERE site.site_name = 'stackoverflow'
	AND account.id = so_user.account_id
	AND site.site_id = so_user.site_id
	AND so_user.id = top100_posters.owner_user_id;
""")


{'select': [{'value': 'account.display_name'},
  {'value': 'top100_posters.n_answers'}],
 'from': ['account', 'so_user', 'site', 'top100_posters'],
 'where': {'and': [{'eq': ['site.site_name', {'literal': 'stackoverflow'}]},
   {'eq': ['account.id', 'so_user.account_id']},
   {'eq': ['site.site_id', 'so_user.site_id']},
   {'eq': ['so_user.id', 'top100_posters.owner_user_id']}]},
 'with': {'name': 'top100_posters',
  'value': {'select': [{'value': 'owner_user_id'},
    {'value': {'count': '*'}, 'name': 'n_answers'}],
   'from': ['answer', 'site'],
   'where': {'and': [{'eq': ['site.site_name', {'literal': 'stackoverflow'}]},
     {'eq': ['answer.site_id', 'site.site_id']}]},
   'groupby': {'value': 'owner_user_id'},
   'orderby': {'value': 'n_answers', 'sort': 'desc'},
   'limit': 100}}}

## Parallel parsing

This does not work b/c MOSP maintains a global parser lock.
